In [ ]:
# Create a Magenta Environment: 

#@title Setup Environment
#@test {"output": "ignore"}

import glob

BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta
!git clone https://github.com/MatthewAwesome/AIComposer

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# A wrapper function to play a note sequence: 
def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

# A function to interpolate two sequences: 
def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print('Start Seq Reconstruction')
  play(note_sequences[0])
  print('End Seq Reconstruction')
  play(note_sequences[-1])
  print('Mean Sequence')
  play(note_sequences[num_steps // 2])
  print('Start -> End Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps >= 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print('Done')

In [ ]:
# Grabbing the models: 
trio_models = {}
hierdec_trio_16bar_config = configs.CONFIG_MAP['hierdec-trio_16bar']
trio_models['hierdec_trio_16bar'] = TrainedModel(hierdec_trio_16bar_config, batch_size=1, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/trio_16bar_hierdec.ckpt')
#
flat_trio_16bar_config = configs.CONFIG_MAP['flat-trio_16bar']
trio_models['baseline_flat_trio_16bar'] = TrainedModel(flat_trio_16bar_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/trio_16bar_flat.ckpt')

In [ ]:
import os
os.chdir('./AIComposer')
songs = []
with open("I Shot the Sheriff.2.mid",'rb') as song1: 
  songs.append([song1.read()])
with open("Band On The Run.mid",'rb') as song2: 
  songs.append([song2.read()])


In [ ]:
# We use two cells here. One to load a first song. 
songs = []
songs.append(files.upload().values() or input_trio_midi_data)


In [ ]:
# Another to load the second song.
songs.append(files.upload().values() or input_trio_midi_data)


In [ ]:
all_trios = []
for i,s in enumerate(songs): 
  trio_input_seqs = [mm.midi_to_sequence_proto(m) for m in s]
  extracted_trios = []
  for ns in trio_input_seqs:
    extracted_trios.extend(
        hierdec_trio_16bar_config.data_converter.from_tensors(
            hierdec_trio_16bar_config.data_converter.to_tensors(ns)[1]))
  all_trios.append(extracted_trios)


In [ ]:
#@title Compute the reconstructions and mean of the two trios, selected from the previous cell.

# hierdec contains an LSTM hierarchy,
# flat does not. 

trio_interp_model = "hierdec_trio_16bar" #@param ["hierdec_trio_16bar", "baseline_flat_trio_16bar"]

song_one_trio =  1#@param {type:"integer"}
song_two_trio =  0#@param {type:"integer"}
start_trio = all_trios[0][song_one_trio]
end_trio = all_trios[1][song_two_trio]

temperature = 1.2 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
trio_16bar_mean = interpolate(trio_models[trio_interp_model], start_trio, end_trio, num_steps=3, max_length=256, individual_duration=32, temperature=temperature)


In [ ]:
download(trio_16bar_mean, 'your_awesome_mashup.mid')